In [5]:
#import torch
#print(torch.__version__)
# !pip uninstall torch -y
# !pip uninstall torchvision -y
# !pip uninstall torchaudio -y

# #!nvcc --version
# !pip install torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0

# !pip install timm==0.4.9 einops

# !pip install apex

# !pip install mmdet

# !pip install matplotlib

# !pip install mmcv-full==1.3.9 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

!git clone https://github.com/ViTAE-Transformer/ViTPose.git
!pip install -v -e ./ViTPose

Cloning into 'ViTPose'...
remote: Enumerating objects: 1737, done.
remote: Counting objects: 100% (1726/1726), done.
remote: Compressing objects: 100% (894/894), done.
remote: Total 1737 (delta 814), reused 1705 (delta 802), pack-reused 11
Receiving objects: 100% (1737/1737), 10.66 MiB | 5.55 MiB/s, done.
Resolving deltas: 100% (817/817), done.
Using pip 22.1.2 from /Users/cristianverdecchia/miniconda3/envs/skeletoncnn/lib/python3.9/site-packages/pip (python 3.9)
Obtaining file:///Users/cristianverdecchia/Documents/University/Master/Advanced%20Computer%20Vision%20%26%20Image%20Processing/human_pose_estimation/SJ-CV/ViTPose
  Preparing metadata (setup.py) ...   Running command python setup.py egg_info
  running egg_info
  creating /private/var/folders/mj/_zlzn07d7xnd1bcr0_w6w2980000gn/T/pip-pip-egg-info-juodrx_m/mmpose.egg-info
  writing /private/var/folders/mj/_zlzn07d7xnd1bcr0_w6w2980000gn/T/pip-pip-egg-info-juodrx_m/mmpose.egg-info/PKG-INFO
  writing dependency_links to /private/var/

In [3]:
# Results from this repo on MS COCO val set (multi-task training)
# MODEL B
!wget -O model_b.pth https://fd0rfg.sn.files.1drv.com/y4mjxZwPM-eGNFmXLsf91PehujeQMrrF_3u34fFj5aaZyJVLka85mkdL8CXELnZUv5dINCzwWk_gtuxjs5DSluasUcio_U1DRqNAnMFeaACjUuYq65nIV7UBzeutRE1OE2HYfkD79eXHaD-VR0r8txxAQot2TQOnafJgBfPQSxgMyak93Gt8_V7fr4WZFJbFy_fasyBRBl1OYh_jOr--167fQ

zsh:1: command not found: wget


In [10]:
import matplotlib.pyplot as plt
from PIL import Image
import copy
import json
import os
import multiprocessing

import numpy as np
from mmpose.apis import (inference_bottom_up_pose_model,
                         inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_pose_result)
from mmpose.datasets import DatasetInfo

ImportError: cannot import name 'container_abcs' from 'torch._six' (/Users/cristianverdecchia/miniconda3/envs/skeletoncnn/lib/python3.9/site-packages/torch/_six.py)

In [8]:
def init_model():
  return init_pose_model(
      './ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/'
      'coco/ViTPose_base_coco_256x192.py',
      'model_b.pth',
      device='cpu')

def init_dataset_info(pose_model):
  dataset_info = DatasetInfo(pose_model.cfg.data['test'].get(
      'dataset_info', None))
    

def vis_top_down(pose_model, dataset_info, person_result, image_path):
  # # test a single image, with a list of bboxes.
  pose_results = get_pose(
      pose_model, dataset_info, person_result, image_path)
  
  # show the results
  return vis_pose_result(
      pose_model, image_path, pose_results, dataset_info=dataset_info)

def get_pose(pose_model, dataset_info, person_result, image_path):
  # # test a single image, with a list of bboxes.
  pose_results, _ = inference_top_down_pose_model(
      pose_model,
      image_path,
      person_result,
      format='xyxy',
      dataset_info=dataset_info)
  return pose_results

In [12]:
from google.colab import files
from google.colab import drive

drive.mount("/content/drive")

ModuleNotFoundError: No module named 'google.colab'

In [9]:
with open('../annotation/annotation_image_info.json') as f:
  annotations = json.load(f)

pose_model = init_model()
dataset_info = init_dataset_info(pose_model)

NameError: name 'init_pose_model' is not defined

In [ ]:
class PostureLink():
  def __init__(self, posture_id, image_detection):
    self.posture_id = posture_id
    self.image_detection = image_detection

  def toJSON(self):
    return json.dumps(self, default=lambda o: o.__dict__,
                     sort_keys=True, indent=4)


class ImageDetectionLink():
  def __init__(self, image_name, relation_ids, person_1, person_2):
    self.image_name = image_name  
    self.relation_ids = relation_ids
    self.person_1 = person_1
    self.person_2 = person_2
  # def add_pose(self, pose):
  #   self.people_poses.append(pose)

  def toJSON(self):
    return json.dumps(self, default=lambda o: o.__dict__,
                     sort_keys=True, indent=4)
    

class PersonDetection():
  def __init__(self, left_eye, right_eye, left_ear, right_ear,  left_shoulder, 
               right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist, 
               left_hip, right_hip, left_knee, right_knee, left_ankle, right_ankle):

    self.left_eye = left_eye.tolist()
    self.right_eye = right_eye.tolist()
    self.left_ear = left_ear.tolist()
    self.right_ear = right_ear.tolist()
    self.left_shoulder = left_shoulder.tolist()
    self.right_shoulder = right_shoulder.tolist()
    self.left_elbow = left_elbow.tolist()
    self.right_elbow = right_elbow.tolist()
    self.left_wrist = left_wrist.tolist()
    self.right_wrist = right_wrist.tolist()
    self.left_hip = left_hip.tolist()
    self.right_hip = right_hip.tolist()
    self.left_knee = left_knee.tolist()
    self.right_knee = right_knee.tolist()
    self.left_ankle = left_ankle.tolist()
    self.right_ankle = right_ankle.tolist()

In [ ]:
#Creates a new detection object (check the config for the meaning of the indexes)
def create_detection(detection):
  return PersonDetection(
      detection[0],
      detection[1],
      detection[2],
      detection[3],
      detection[4],
      detection[5],
      detection[6],
      detection[7],
      detection[8],
      detection[9],
      detection[10],
      detection[11],
      detection[12],
      detection[13],
      detection[14],
      detection[15]
      )

In [ ]:
directory = '/content/drive/MyDrive/AdvancedComputerVision/image/'

#list_of_files_from_folder = [s.lstrip("0") for s in sorted(os.listdir(directory))]

# Estimates poses and creates Detection objects for each pose
def detect_poses(file_annotations):
  boxes = []

  file_name = file_annotations[1]['file_name']
  # Transform values inside to int
  bblist = list(map(int, file_annotations[1]['pose']))

  #  image_id = int(image_name[:-4])
  image_id = int(file_name)

  file_name = file_name + ".jpg"
  for x in range(5 - len(file_name[:-4])):
    file_name = "0" + file_name

  # Create an object with the file and the list of poses
  for annotation in annotations:
      if annotation['id'] == image_id:
          # The -1 is required for the index since the indexes represent the 
          # relations between people ex: person 1 and person 2, there's no person 0
          boxes.append({'bbox': annotation['bbox'][bblist[0]-1]})
          boxes.append({'bbox': annotation['bbox'][bblist[1]-1]})
          print(file_name+" annotation ID: "+str(image_id))

  # Obtain poses for each bbox
  obtained_poses = get_pose(pose_model, dataset_info, boxes, directory+file_name)
  
  first_person = create_detection(obtained_poses[0]['keypoints'][:,:-1])
  second_person = create_detection(obtained_poses[1]['keypoints'][:,:-1])

  image_detection = ImageDetectionLink(image_id, bblist, first_person, second_person)
  posture_link = {file_annotations[0] : image_detection}
  
  return posture_link

In [ ]:
# TODO CHANGE IT FOR COUPLE OF POSES INSTEAD OF FILES
# file_dict = {1:[], 2:[],3:[], 4:[], 5:[],6:[]}
# for file_name in relationships.keys():
#   for people_link in relationships[file_name]:
#     if file_name not in file_dict[relationships[file_name][people_link]]:
#       file_dict[relationships[file_name][people_link]].append(str(file_name))

In [ ]:
with open('/content/drive/MyDrive/AdvancedComputerVision/annotation/relationship.json') as f:
  relationships = json.load(f)

In [ ]:
boxes = {1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}

# Loop all the files and relationships
for file_id, dict_link in relationships.items():
    # Loop the relationships with labels
    for linking, label in dict_link.items():
      indexes = linking.split()
      # Check if the labels have already been added
      if file_id not in boxes[label]:
        boxes[label].update({file_id: [indexes]})
      else:
        boxes[label][file_id].append(indexes)

print(boxes)

In [ ]:
import random
from itertools import islice

# # Creates a subset for a given label
# def get_label_data(index, amount):    
#   label = islice(
#       boxes[index].items(), amount)
#   return dict(label)

# label_1 = get_label_data(1, 500)
# label_2 = get_label_data(2, 500)
# label_3 = get_label_data(3, 500)
# label_4 = get_label_data(4, 500)
# label_5 = get_label_data(5, 500)
# label_6 = get_label_data(6, 500)

In [ ]:
# Saves the full annotations

label_1 = dict(boxes[1].items())
label_2 = dict(boxes[2].items())
label_3 = dict(boxes[3].items())
label_4 = dict(boxes[4].items())
label_5 = dict(boxes[5].items())
label_6 = dict(boxes[6].items())

In [ ]:
class LabelData():
  def __init__(self, label_1, label_2, label_3, label_4, label_5, label_6):
    self.label_1 = label_1
    self.label_2 = label_2
    self.label_3 = label_3
    self.label_4 = label_4
    self.label_5 = label_5
    self.label_6 = label_6

In [ ]:
labels_to_save = LabelData(label_1, label_2, label_3, label_4, label_5, label_6)

with open("/content/drive/MyDrive/AdvancedComputerVision/annotation/labels_with_files_s_500.json", "w") as outfile:
    json.dump(labels_to_save, outfile, default=lambda o: o.__dict__ )

In [ ]:
import json

with open('/content/drive/MyDrive/AdvancedComputerVision/annotation/labels_with_files_s_500.json') as f:
  file_annotations = json.load(f)

In [ ]:
label_dict = {1:{}, 2:{}, 3:{}, 4:{}, 5:{}, 6:{}}
index = 0

# Loop through the labels
for label, value in enumerate(file_annotations.values()):
  # Loop each file
  for link in value.items():
    for pose in link[1]:
      # Add the file and the poses to the dict
      inner_dict = {'file_name': link[0], 'pose': pose}
      # Update the dict with file name and poses
      temp_dict = {index : inner_dict}
      label_dict[label+1].update(temp_dict)
      index = index + 1

# for label, stuff in enumerate(label_dict.values()):
#   print(len(stuff))

In [ ]:
print("Images per label")
# Amount of images per label
for label, value in enumerate(file_annotations.values()):
  print(label+1, len(value))

print("Poses per label")
# Amount of poses per label
for label, value in enumerate(label_dict.values()):
  print(label+1, len(value.items()))

Images per label
1 5129
2 3733
3 1890
4 7308
5 673
6 2439
Poses per label
1 13808
2 8744
3 1910
4 22011
5 1041
6 13352


In [ ]:
from itertools import islice

# Transform the dict into chunks (does work but not properly)
# This has to be used just for tests purposes
# def chunks(data, SIZE=10000):
#     it = iter(data)
#     for i in range(0, len(data), SIZE):
#         yield {k:data[k] for k in islice(it, SIZE)}
# # Put the samples into a dict
# for item in chunks(label_dict[1], 10):
#   test = item
# print(len(test))


In [ ]:
# def get_label_data_to_run(index, amount):    
#   label = islice(
#       label_dict[index].items(), amount)
#   return dict(label)

# print(len(get_label_data(1, 500).items()))

NameError: ignored

In [ ]:
label_to_run = 5

# label_dict_to_run = get_label_data_to_run(label_to_run, 500)
label_dict_to_run = label_dict[label_to_run]
print(len(label_dict_to_run))

#Parallelise the process, so to compute the same for multiple images
multiprocessing.set_start_method('fork', force=True)
pool = multiprocessing.Pool(int(os.cpu_count()))

# Uses all of the available labels
image_detection_list = pool.map(detect_poses, label_dict_to_run.items())

# Uses only few examples to test the functionality
# image_detection_list = pool.map(detect_poses, test.items())
pool.close()


#{image_name: 'im.jpg', body_poses=[['leftknee'..., label=1]['right_knee'...,label=2]]}}

print("Execution completed")

In [ ]:
final_dict = {}

for image_detection in image_detection_list:
  image_id = list(image_detection.keys())[0]
  final_dict[image_id] = list(image_detection.values())[0]

In [ ]:
with open("/content/drive/MyDrive/AdvancedComputerVision/posture_data/"+str(label_to_run)+"_full_bb.json", "w") as outfile:
        json.dump( final_dict, outfile, default=lambda o: o.__dict__ )

In [ ]:
len(image_detection_list)